# Quick Map
  

In [1]:
import sys;sys.path.append('../html-maps')
from mapfun import *
from features import *
import pathlib as pl
import geopandas as gpd
from IPython.display import IFrame

### Read in data, projections, etc.

In [2]:
rootdir = pl.Path(r'Y:\CCSI\TECH\FEMA\2019_PFRA\DATA\NJ\Spatialfiles\Scope\html_map')
crs = '+proj=aea +lat_1=20 +lat_2=60 +lat_0=40 +lon_0=-96 +x_0=0 +y_0=0 +datum=NAD83 +units=m +no_defs' 
name = 'New York'

gages = gpd.read_file(str(rootdir/'gages.geojson'))
gages = quick_transform(gages, crs)

rivers = gpd.read_file(str(rootdir/'rivers.geojson'))
rivers = quick_transform(rivers, crs)

huc12s = gpd.read_file(str(rootdir/'huc12s.geojson'))
huc12s = quick_transform(huc12s, crs)

### Assign Map Center

In [3]:
xcoord, ycoord = get_centroid(gages)
m = interactive_map(ycoord, xcoord)

### For each shape added to the map, assign columnar data to show in pop-up

In [4]:
gages_info = {'Gage ID':'Gage_ID', 'Station':'Station_NM', 'URL':'NWISWeb'}
header= ['Company','Dewberry']
title = ''
params = column_styles(2,headers=header,size='55%',col_font='font-size: 10pt')
gages_descr = table_description(gages, params, gages_info, '<TH style="font-size:10pt;text-align:center">',
                                 title,change_header=True)
m = add_point_tables(gages, m, gages_descr, gages['Gage_ID'])

### Make popup for lines

In [5]:
rivers_info = {'REACH ID':'REACH_ID'}
header= ['Company','Dewberry']
title = ''
params = column_styles(2,headers=header,col_font='font-size: 10pt')
rivers_descr = table_description(rivers, params,rivers_info, '<TH style="font-size:10pt;text-align:center">',
                                 title,change_header=True)
m = add_multiline(rivers, m, rivers_descr, rivers['REACH_ID'])

### Make popup for polygons

In [6]:
huc12s_info = {'HUC12':'HUC12', 'Domain':'Domains', 'Label':'Label', 'PM':'PM'}
title = f'<caption style="font-size:14pt;text-align:center"><caption>{name}</caption><TH>'
header= ['Company',huc12s['Company'][0]]
params = column_styles(2,headers=header,size='60%',col_font='font-size: 10pt')
descriptions = table_description(huc12s, params, huc12s_info, '<TH style="font-size:10pt;text-align:center">',
                                 title,change_header=False)
m = add_poly_table(huc12s, m,descriptions, huc12s['Label_1'])

### Add extra features to map

In [7]:
m = add_tiles(m,tile_type='World_Light_Gray_Base',services='/Canvas')
m = add_extra(m)

### Save to html

In [8]:
m.save(outfile='map.html')  
IFrame('map.html', width=800, height=650)

# END